# Investing On ETFs Using The Kelly Formula Part 2

## How To Allocate Capital Using The Kelly Formula

## Introduction
...

## The Kelly Formula

### Ed Thorp
...

### Optimal Capital Allocation
...

## Menu
1. ...
2. ...

Import packages

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import pickle

Pickle

In [2]:
# # Set files to pickle
# input = ['etf_us_bonds.pickle',
#          'etf_us_stocks.pickle',
#          'etf_intl_bonds.pickle',
#          'etf_intl_stocks.pickle',
#          'etf_sectors.pickle']

# # Pickle 
# for i in input:
#     with open(i, 'rb') as f:
#         data = pickle.load(f)
#     f.close()
    


Pickle

In [48]:
with open('etf_us_bonds.pickle', 'rb') as f:
    
    data = pickle.load(f)
    
f.close()

In [52]:
data.columns.levels[0]

Index(['BIV', 'BLV', 'BND', 'BNDW', 'BNDX', 'BSV', 'EDV', 'ESGV', 'MGC', 'MGK',
       'MGV', 'VAW', 'VB', 'VBK', 'VBR', 'VCIT', 'VCLT', 'VCR', 'VCSH', 'VDC',
       'VDE', 'VEA', 'VEU', 'VFH', 'VGIT', 'VGK', 'VGLT', 'VGSH', 'VGT', 'VHT',
       'VIG', 'VIGI', 'VIS', 'VMBS', 'VNQ', 'VNQI', 'VO', 'VOE', 'VOO', 'VOT',
       'VOX', 'VPL', 'VPU', 'VSGX', 'VSS', 'VT', 'VTC', 'VTEB', 'VTI', 'VTIP',
       'VTV', 'VUG', 'VV', 'VWO', 'VWOB', 'VXF', 'VXUS', 'VYM', 'VYMI'],
      dtype='object')

Create a DataFrame object of close prices

Drop NaNs

In [8]:
data.dropna(inplace=True)

Calculate daily returns

In [9]:
returns = data.pct_change()

Risk-free rate 

In [10]:
risk_free = 0.025

Excess daily returns

In [11]:
excess_returns = returns - risk_free

Mean of excess daily returns annualized

In [12]:
excess_returns

Empty DataFrame
Columns: [(EDV, open), (EDV, high), (EDV, low), (EDV, close), (EDV, volume), (BIV, open), (BIV, high), (BIV, low), (BIV, close), (BIV, volume), (VGIT, open), (VGIT, high), (VGIT, low), (VGIT, close), (VGIT, volume), (BLV, open), (BLV, high), (BLV, low), (BLV, close), (BLV, volume), (VGLT, open), (VGLT, high), (VGLT, low), (VGLT, close), (VGLT, volume), (VMBS, open), (VMBS, high), (VMBS, low), (VMBS, close), (VMBS, volume), (BSV, open), (BSV, high), (BSV, low), (BSV, close), (BSV, volume), (VTIP, open), (VTIP, high), (VTIP, low), (VTIP, close), (VTIP, volume), (VGSH, open), (VGSH, high), (VGSH, low), (VGSH, close), (VGSH, volume), (BND, open), (BND, high), (BND, low), (BND, close), (BND, volume), (VCIT, open), (VCIT, high), (VCIT, low), (VCIT, close), (VCIT, volume), (VCLT, open), (VCLT, high), (VCLT, low), (VCLT, close), (VCLT, volume), (VCSH, open), (VCSH, high), (VCSH, low), (VCSH, close), (VCSH, volume), (VTC, open), (VTC, high), (VTC, low), (VTC, close), (VTC, volume), (VTEB, open), (VTEB, high), (VTEB, low), (VTEB, close), (VTEB, volume), (VIG, open), (VIG, high), (VIG, low), (VIG, close), (VIG, volume), (ESGV, open), (ESGV, high), (ESGV, low), (ESGV, close), (ESGV, volume), (VUG, open), (VUG, high), (VUG, low), (VUG, close), (VUG, volume), (VYM, open), (VYM, high), (VYM, low), (VYM, close), (VYM, volume), (VV, open), (VV, high), (VV, low), (VV, close), (VV, volume), ...]
Index: []

[0 rows x 295 columns]

## Conclusion
...